# Breast cancer case

In [34]:
import numpy as np
import pandas as pd
import feyn

from sklearn.model_selection import train_test_split, StratifiedKFold


import matplotlib.pyplot as plt

from _crossvalidation import crossvalidation_as_framework

### Load and prep data

In [2]:
data = pd.read_csv('../data/brca_data_w_meta.csv')

In [3]:
# Define the target variable
target = "vital.status"

In [4]:
stypes = {}
for f in data.columns:
    if data[f].dtype == 'object':
        stypes[f] = 'c'

### 2 feature models

In [23]:
results = crossvalidation_as_framework(data,
                                     target,
                                     kind = "classification",
                                     stypes = stypes,
                                     n_epochs = 100,
                                     criterion='bic',
                                     max_complexity = 3)

KeyboardInterrupt: 

In [12]:
results.groupby("fold").first().roc_auc_val.mean(), results.roc_auc_val.mean()

(0.6183480214152156, 0.6313666034560834)

In [17]:
for model in results.groupby("fold").first().model_structure:
    print(model)

logreg((cnCLDN19 + muGPR98)*(rsCIDEC*rsPIK3C2G + rsSLC7A4))
logreg((rsALOX15 + rsIGSF1)*(rsALOX15B + rsAPOB + rsTRPV6))
logreg(tanh(rsKLK8*(muPDZD2 + rsTMPRSS4 + exp(-cnSLC13A2**2 - rsLGALS12**2))))
logreg(cnTGFBR3*rsPOF1B + rsFGFBP1*(rsSLC6A15 + tanh(rsCIDEC)))
logreg(tanh(cnABCC6 + rsMAGEA3 + rsSLC28A3*(rsCACNG4 + rsGLYAT)))


### Larger models (default options)

In [18]:
results100 = crossvalidation_as_framework(data,
                                     target,
                                     kind = "classification",
                                     stypes = stypes,
                                     n_epochs = 100,
                                     criterion='bic')

In [22]:
results.groupby("fold").first().roc_auc_val.mean(), results.roc_auc_val.mean()

(0.6183480214152156, 0.6313666034560834)

In [19]:
for model in results100.groupby("fold").first().model_structure:
    print(model)

logreg(rsPTCHD1*(cnADAMTS16 + muDMD + rsFABP4 + rsSOX2))
logreg(rsAPOB*(cnPRODH*(cnCOL14A1 + ppPI3K.p110.alpha) + rsTAT))
logreg(cnCLCA2 + rsSLC13A2*(cnSLC30A8 + rsAPOB + rsPAX7))
logreg(rsTCN1*(ppp27 + rsDEFB132*rsPTPRZ1 + rsSOX2))
logreg(cnTNFRSF11B + ppMSH6**2 + rsCYP4Z2P + rsPCOLCE2 + rsPLA2G2D)


#### Exploration of best models

In [35]:
ql = feyn.connect_qlattice()
ql.reset(random_seed=42)


In [ ]:
best_models_qstrings = ['"rs_PTCHD1"*("cn_ADAMTS16" + "mu_DMD" + "rs_FABP4" + "rs_SOX2")',
                        '"rs_APOB"*("cn_PRODH"*("cn_COL14A1" + "pp_PI3K.p110.alpha") + "rs_TAT")',
                        '"cn_CLCA2" + "rs_SLC13A2"*("cn_SLC30A8" + "rs_APOB" + "rs_PAX7")',
                        '"rs_TCN1"*("pp_p27" + "rs_DEFB132"*"rs_PTPRZ1" + "rs_SOX2")',
                        '"cn_TNFRSF11B" + squared("pp_MSH6") + "rs_CYP4Z2P" + "rs_PCOLCE2" + "rs_PLA2G2D"']
n_folds = 5
random_state = 42
models_trained = []
train_list = []
val_list = []

kfold_test = StratifiedKFold(n_folds, shuffle=True, random_state=random_state)
for i, (train, val) in enumerate(kfold_test.split(data, data[target])):
        train, val = data.iloc[train], data.iloc[val]
        train_list.append(train)
        val_list.append(val)
        
        sample_weights = np.where(train[target] == 1,
                                      np.sum(train[target] == 0) / sum(train[target]), 1)
        
        model = ql.auto_run(train, output_name=target, n_epochs=10, kind = "classification",
                             query_string=best_models_qstrings[i], sample_weights=sample_weights, stypes=stypes)[0]
        models_trained.append(model)
        

In [ ]:
models_trained[0].plot(train_list[0], val_list[0])

### Comparison with random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
list_auc = []
n_folds = 5
random_state = 42
df = pd.get_dummies(data)

kfold_test = StratifiedKFold(n_folds, shuffle=True, random_state=random_state)


for i, (train, val) in enumerate(kfold_test.split(df, df[target])):
    train, val = df.iloc[train], df.iloc[val]
    params = {'bootstrap': True,
     'ccp_alpha': 0.0,
     'class_weight': 'balanced',
     'max_depth': 4,
     'max_features': 'auto',
     'max_leaf_nodes': None,
     'max_samples': None,
     'min_impurity_decrease': 0.0,
     'min_samples_leaf': 1,
     'min_samples_split': 2,
     'min_weight_fraction_leaf': 0.0,
     'n_estimators': 100,
     'random_state': 42
    }

    rf = RandomForestClassifier(**params)
    rf.fit(train.drop(columns=target), train[target])
    preds = rf.predict(val.drop(columns=target))

    list_auc.append(roc_auc_score(val[target], preds))